# Experiment 3: Zero-Shot Pollen Reconstruction with Hunyuan-3D-2

In [2]:
import pandas as pd

In [6]:
results = pd.read_csv("../eval/TestEvaluation/mesh_eval_results.csv")

# print all models names unique
print("Models names unique:")
print(results["model"].unique())

Models names unique:
['pix2vox_aug' 'pix2vox_aug_1img' 'pix2vox_aug_3img' 'pix2vox_aug_4img'
 'pix2vox_aug_5img' 'pix2vox_aug_6img' 'pix2vox_aug_frozen_encdec'
 'pix2vox_base' 'pix2vox_non_pretrained' 'vh_1img' 'vh_2img' 'vh_3img'
 'vh_4img' 'vh_5img' 'vh_6img' 'coarse_mvp2m_2_inputs_none_augmentation'
 'coarse_mvp2m_augmentation_2_inputs'
 'coarse_mvp2m_augmentation_2_inputs_freezed'
 'coarse_mvp2m_augmentation_2_inputs_pretrained'
 'coarse_mvp2m_augmentation_2_inputs_use_mean_prior'
 'coarse_mvp2m_augmentation_2_inputs_use_spherical_prior'
 'coarse_mvp2m_augmentation_3_inputs' 'coarse_mvp2m_augmentation_4_inputs'
 'coarse_mvp2m_augmentation_5_inputs' 'coarse_mvp2m_augmentation_6_inputs'
 'Hunyuan3D' 'Hunyuan3D-four-views' 'Hunyuan3D-two-views'
 'Pixel2Mesh_Augmented_2_Views' 'pollen_128_2' 'pollen_256_2'
 'pollen_augmentation1' 'pollen_augmentation2' 'pollen_augmentation3'
 'pollen_augmentation4' 'pollen_augmentation6'
 'pollen_augmentation_finetune' 'pollen_augmentation_freezed'
 'p

In [7]:

relevant_models = [
    "Hunyuan3D-two-views",
    "Hunyuan3D-four-views",
    "Hunyuan3D",
    # extend with Pixel2Mesh++ and PixelNeRF names as in the CSV file
]

filtered = results[results["model"].isin(relevant_models)]

aggregated = filtered.groupby("model", as_index=False).agg(
    mean_chamfer         = ("chamfer",     "mean"),
    std_chamfer          = ("chamfer",     "std"),
    mean_fscore_1        = ("fscore_1",    "mean"),
    std_fscore_1         = ("fscore_1",    "std"),
    mean_fscore_2_5      = ("fscore_2_5",  "mean"),
    std_fscore_2_5       = ("fscore_2_5",  "std"),
    mean_fscore_5        = ("fscore_5",    "mean"),
    std_fscore_5         = ("fscore_5",    "std"),
    mean_iou             = ("voxel_iou",   "mean"),
    std_iou              = ("voxel_iou",   "std"),
)

aggregated

,model,mean_chamfer,std_chamfer,mean_fscore_1,std_fscore_1,mean_fscore_2_5,std_fscore_2_5,mean_fscore_5,std_fscore_5,mean_iou,std_iou
0,Hunyuan3D,0.049375,0.014329,0.208953,0.098090,0.675685,0.137996,0.889090,0.086445,0.749712,0.118144
1,Hunyuan3D-four-views,0.054040,0.026377,0.209615,0.089645,0.667568,0.157024,0.869052,0.120266,0.734940,0.141448
2,Hunyuan3D-two-views,0.061182,0.033339,0.182319,0.082814,0.607564,0.160457,0.845227,0.130265,0.707719,0.133846
